In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, UpSampling2D
from keras.metrics import BinaryAccuracy
from keras.models import Model

from sklearn.neighbors import NearestNeighbors

import cv2
import os
import numpy as np
from imutils import build_montages

from google.colab.patches import cv2_imshow

from tqdm import tqdm
import shutil

In [ ]:
!mkdir /content/images
!mkdir /content/images/train
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/train
!mkdir /content/labels/test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q '/content/drive/My Drive/NATO/train.zip' -d '/content/images/train'
!unzip -q '/content/drive/My Drive/NATO/train_l.zip' -d '/content/labels/train'
!unzip -q '/content/drive/My Drive/NATO/test.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test_l.zip' -d '/content/labels/test'

In [ ]:
Dict = {0: 'advance_to_contact', 2: 'attack', 9: 'counterattack', 18: 'main_attack'}

In [ ]:
Dict_images = {key: [] for key in Dict.values()}
Dict_images_test = {key: [] for key in Dict.values()}

In [ ]:
images = '/content/images/train'
labels = '/content/labels/train'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (120, 120)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            try:
                Dict_images[Dict[int(line.split(" ")[0])]].append(resized_img)
            except KeyError:
                pass

In [ ]:
images = '/content/images/test'
labels = '/content/labels/test'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (120, 120)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            try:
                Dict_images_test[Dict[int(line.split(" ")[0])]].append(resized_img)
            except KeyError:
                pass

In [ ]:
def model_version1():
    # autoencoder model
    # encoder
    input = Input(shape=(120, 120, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D(pool_size=(2, 2), padding='same', name="encoded")(x)
    #decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(inputs=input, outputs=decoded)

    return model

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    x_train = np.array(Dict_images[str(i)])
    x_train = x_train.astype('float32') / 255.
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

    x_test = np.array(Dict_images_test[str(i)])
    x_test = x_test.astype('float32') / 255.
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    print(x_train.shape, x_test.shape)

    # autoencoder model
    # encoder
    input = Input(shape=(120, 120, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D(pool_size=(2, 2), padding='same', name="encoded")(x)
    #decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(inputs=input, outputs=decoded)

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Fit the model
    model.fit(x_train, x_train, batch_size = 64, epochs = 50, validation_data=(x_test, x_test))

    model.save(str(i)+"_trajectory_autoencoder.h5")

In [ ]:
for i in Dict_images.keys():
    colab_link = '/content/'+str(i)+'_trajectory_autoencoder.h5'
    gdrive_link = "/content/drive/My Drive/models_trajectory/"
    shutil.copy(colab_link, gdrive_link)

In [ ]:
for i in Dict_images_test.keys():
    str(i)
    model.load_weights('/content/drive/MyDrive/models_trajectory/'+str(i)+'_trajectory_autoencoder.h5')

    images = np.array(Dict_images_test[str(i)])
    images = images.astype('float32') / 255.
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

    decoded_images = model.predict(images)

    number_of_images = list(range(0, images.shape[0]))
    random_images = np.random.choice(number_of_images, size=2, replace=False)

    examples = []
    for i in random_images:
        original = images[i] * 255
        decoded_img = decoded_images[i] * 255
        comparison = np.hstack([original, decoded_img])
        examples.append(comparison)

    comparsion_images = np.vstack(examples)
    cv2_imshow(comparsion_images)

In [ ]:
for i in Dict_images.keys():
    str(i)
    model.load_weights('/content/drive/MyDrive/models_trajectory/'+str(i)+'_trajectory_autoencoder.h5')
    encoder = Model(inputs=model.input, outputs=model.get_layer("encoded").output)

    images = np.array(Dict_images[str(i)])
    images = images.astype('float32') / 255.
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

    features = encoder.predict(images)
    indexes = list(range(0, images.shape[0]))
    data = {"indexes": indexes, "images": Dict_images[str(i)], "features": features}
    np.save(str(i)+"_trajectory_data.npy", data)

In [ ]:
for i in Dict_images.keys():
    colab_link = '/content/'+str(i)+'_trajectory_data.npy'
    gdrive_link = "/content/drive/My Drive/models_trajectory/"
    shutil.copy(colab_link, gdrive_link)

## Lõpp test sümbolid

In [ ]:
!mkdir /content/images
!mkdir /content/images/test_trajectory
!mkdir /content/labels
!mkdir /content/labels/test_trajectroy

!unzip -q '/content/drive/My Drive/NATO/test_trajectory.zip' -d '/content/images/test_trajectory'
!unzip -q '/content/drive/My Drive/NATO/test_trajectory_l.zip' -d '/content/labels/test_trajectory'

In [ ]:
Dict_images_test_trajectory = {key: [] for key in Dict.values()}
images = '/content/images/test_trajectory'
labels = '/content/labels/test_trajectory'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            dim = (120, 120)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            try:
                Dict_images_test_trajectory[Dict[int(line.split(" ")[0])]].append(resized_img)
            except KeyError:
                pass

In [ ]:
model = model_version1()

symbol_valik = "attack"

model.load_weights('/content/drive/MyDrive/models_trajectory/'+symbol_valik+'_trajectory_autoencoder.h5')
encoder = Model(inputs=model.input, outputs=model.get_layer("encoded").output)

images = np.array(Dict_images_test_trajectory[symbol_valik])
images = images.astype('float32') / 255.
images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

predicted = encoder.predict(images)

## Find similarity

In [ ]:
number_of_images = list(range(0, images.shape[0]))
random_images = np.random.choice(number_of_images, size=3, replace=False)

In [ ]:
load_data = np.load("/content/"+symbol_valik+"_trajectory_data.npy", allow_pickle=True)
load_features = load_data.item().get("features")
load_images = load_data.item().get("images")
features_reshape = load_features.reshape((-1, np.prod((load_features.shape[1:]))))

## KNN (Cosine)

In [ ]:
knn_cosine = NearestNeighbors(n_neighbors=5, metric="cosine")
knn_cosine.fit(features_reshape)

In [ ]:
predicted_reshape = predicted.reshape((-1, np.prod((predicted.shape[1:]))))

for i in random_images:
    distances, indices = knn_cosine.kneighbors([predicted_reshape[i]])
    results = [load_images[idx] for idx in indices.flatten()]
    imgs = []
    for index in results:
        image = index.astype("uint8")
        image = np.dstack([image] * 3)
        imgs.append(image)
    query = (images[i] * 255).astype("uint8")
    print("Original")
    cv2_imshow(query)
    montage = build_montages(imgs, (70, 70), (5, 1))[0]
    print("Similar symbols")
    cv2_imshow(montage)

## KNN (Euclidean)

In [ ]:
knn_euclidean = NearestNeighbors(n_neighbors=5, metric="euclidean")
knn_euclidean.fit(features_reshape)

In [ ]:
predicted_reshape = predicted.reshape((-1, np.prod((predicted.shape[1:]))))

for i in random_images:
    distances, indices = knn_euclidean.kneighbors([predicted_reshape[i]])
    results = [load_images[idx] for idx in indices.flatten()]
    imgs = []
    for index in results:
        image = index.astype("uint8")
        image = np.dstack([image] * 3)
        imgs.append(image)
    query = (images[i] * 255).astype("uint8")
    print("Original")
    cv2_imshow(query)
    montage = build_montages(imgs, (70, 70), (5, 1))[0]
    print("Similar symbols")
    cv2_imshow(montage)